## 1. Setup

Import necessary libraries and define the path to the configuration file.

# YOLOv8 Training Notebook\\n"
This notebook trains a YOLOv8 model using a configuration defined in `config.yaml`

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# YOLOv8 Training Script
#
# This script trains a YOLOv8 model using a configuration defined in `config.yaml`.
# It replicates the steps typically found in a Jupyter Notebook for training.

# =============================================================================
# ## 1. Setup
#
# Import necessary libraries and define the path to the configuration file.
# =============================================================================
import yaml
import os
from ultralytics import YOLO
import torch # Optional: Check GPU availability
import json # For potentially printing config nicely

CONFIG_PATH = "config.yaml"

# =============================================================================
# ## 2. Load Configuration
#
# Load training parameters from the `config.yaml` file.
# =============================================================================
config = None
dataset_yaml = None
model_type = None
hyperparams = {}
output_config = {}
device = None

print("--- Loading Configuration ---")
try:
    with open(CONFIG_PATH, 'r') as f:
        config = yaml.safe_load(f)
except FileNotFoundError:
    print(f"Error: Configuration file not found at {CONFIG_PATH}")
    # Exit or provide default values if needed
    exit(1) # Exit if config is essential
except Exception as e:
    print(f"Error loading configuration: {e}")
    exit(1) # Exit on other loading errors

if config:
    print("Configuration loaded successfully:")
    # Optional: Print loaded config for verification
    # print(json.dumps(config, indent=2))

    # --- Extract key parameters ---
    dataset_yaml = config.get('dataset_yaml_path')
    model_type = config.get('model_type')
    hyperparams = config.get('hyperparameters', {}) # Default to empty dict
    output_config = config.get('output', {})       # Default to empty dict
    device = config.get('device')

    # --- Basic Validation ---
    valid_config = True
    if not dataset_yaml or not os.path.exists(dataset_yaml):
        print(f"Error: Dataset YAML path '{dataset_yaml}' is invalid or file does not exist. Please check config.yaml.")
        valid_config = False
    if not model_type:
        print("Error: 'model_type' not specified in config.yaml.")
        valid_config = False
    if not output_config.get('project_name') or not output_config.get('experiment_name'):
        print("Error: 'project_name' or 'experiment_name' missing in output configuration.")
        valid_config = False

    if not valid_config:
        print("Configuration errors detected. Exiting.")
        exit(1)
else:
    # This case should ideally be caught by the exit(1) above, but included for completeness
    print("Configuration could not be loaded. Exiting.")
    exit(1)



In [ ]:
# =============================================================================
# ## 3. Environment Check (Optional)
#
# Verify GPU availability if specified.
# =============================================================================
print("\n--- Environment Check ---")
if device and device != 'cpu':
    try:
        print(f"Attempting to use device: {device}")
        gpu_available = torch.cuda.is_available()
        print(f"PyTorch CUDA available: {gpu_available}")
        if not gpu_available:
             print(f"Warning: Device set to '{device}' but CUDA is not available. Training might default to CPU or fail.")
             # Optionally force device to 'cpu' or exit if GPU is mandatory
             # device = 'cpu'
             # print("Forcing device to CPU.")
        else:
            print(f"CUDA Device Count: {torch.cuda.device_count()}")
            # You could add more specific checks for the requested device ID here
            # e.g., check if device string is '0', '1', etc. and if that device exists
    except Exception as e:
        print(f"Error during GPU check: {e}")
else:
    print(f"Device specified as: {device if device else 'Default (likely CPU if no GPU)'}")


# =============================================================================
# ## 4. Initialize Model
#
# Load the specified YOLOv8 model. This could be a pre-trained model
# (like `yolov8n.pt`) or a path to a checkpoint from previous training.
# =============================================================================
print("\n--- Initializing Model ---")
model = None
try:
    print(f"Initializing model: {model_type}")
    model = YOLO(model_type)
    print("Model initialized successfully.")
except Exception as e:
    print(f"Error initializing YOLO model: {e}")
    print("Exiting.")
    exit(1)

In [ ]:
# =============================================================================
# ## 5. Start Training
#
# Run the YOLOv8 training process using the loaded configuration.
# =============================================================================
print("\n--- Starting Training ---")
results = None
if model: # Redundant check if exits above are used, but safe
    try:
        # Prepare keyword arguments, filtering out None values if necessary
        train_args = {
            'data': dataset_yaml,
            'epochs': hyperparams.get('epochs', 100), # Default if not in config
            'imgsz': hyperparams.get('image_size', 640),
            'batch': hyperparams.get('batch_size', 16),
            'project': output_config.get('project_name'),
            'name': output_config.get('experiment_name'),
            'device': device,
            # Add other hyperparameters dynamically
            **{k: v for k, v in hyperparams.items() if k not in ['epochs', 'image_size', 'batch_size'] and v is not None}
        }
        print("Training arguments:")
        print(json.dumps(train_args, indent=2))

        results = model.train(**train_args)

        print("\nTraining finished.")
        # Results object might not always have save_dir if training interrupted early
        if hasattr(results, 'save_dir'):
            print(f"Results saved to: {results.save_dir}")
        else:
             print("Training completed, but couldn't access results.save_dir (may have been interrupted or an older ultralytics version behavior). Check project directory.")
             print(f"Project: {output_config.get('project_name')}, Name: {output_config.get('experiment_name')}")


    except Exception as e:
        print(f"An error occurred during training: {e}")
        # Consider logging the full traceback here for debugging
        import traceback
        traceback.print_exc()
else:
    # Should not be reachable if exits are used correctly
    print("Skipping training due to model initialization failure.")


In [ ]:

# =============================================================================
# ## 6. Evaluation (Optional)
#
# Evaluate the best performing model checkpoint on the validation or test set.
# =============================================================================
print("\n--- Evaluation ---")
if results and hasattr(results, 'save_dir'):
    # Path to the best model weights saved during training
    best_model_path = os.path.join(results.save_dir, 'weights/best.pt')

    if os.path.exists(best_model_path):
        print(f"Loading best model from: {best_model_path}")
        try:
            best_model = YOLO(best_model_path)

            print("\nEvaluating on the validation set...")
            # The 'val' split is typically defined in your data.yaml
            # Ensure device is passed if needed for consistency
            val_metrics = best_model.val(split='val', device=device)
            print("Validation Metrics obtained.")
            # Accessing specific metrics might depend on the task (detect, segment, classify)
            # Example for detection:
            if hasattr(val_metrics, 'box'):
                 print(f"Validation mAP50-95: {val_metrics.box.map}")
                 print(f"Validation mAP50: {val_metrics.box.map50}")

            # --- Evaluate on Test Set (if defined in data.yaml) ---
            # Check if 'test' split exists in the dataset yaml
            # Accessing model.trainer.args.data is one way, but might change or not exist if only loaded
            data_cfg_path = dataset_yaml # Use the path from our config
            test_split_exists = False
            test_path = None
            if isinstance(data_cfg_path, str) and os.path.exists(data_cfg_path):
                try:
                    with open(data_cfg_path, 'r') as f:
                        d_yaml = yaml.safe_load(f)
                        if 'test' in d_yaml and d_yaml['test']:
                            test_path = d_yaml['test']
                            # Basic check if the path seems valid (not exhaustive)
                            if isinstance(test_path, str) and test_path.strip():
                                test_split_exists = True
                                print(f"Test split found in {data_cfg_path}: {test_path}")
                            else:
                                print(f"Test split key found in {data_cfg_path}, but the value is empty or invalid.")
                        else:
                             print(f"'test' key not found or empty in {data_cfg_path}.")

                except Exception as e:
                     print(f"Could not read or parse data yaml {data_cfg_path} to check for test set: {e}")
            else:
                 print(f"Dataset YAML path '{data_cfg_path}' not found, cannot check for test set.")


            if test_split_exists:
                 print("\nEvaluating on the test set...")
                 try:
                    # Ensure device is passed if needed
                    test_metrics = best_model.val(split='test', device=device)
                    print("Test Metrics obtained.")
                    # Example for detection:
                    if hasattr(test_metrics, 'box'):
                        print(f"Test mAP50-95: {test_metrics.box.map}")
                        print(f"Test mAP50: {test_metrics.box.map50}")
                 except Exception as e:
                     print(f"Error during test set evaluation: {e}")
                     # This might happen if the test set path in yaml is invalid or data is corrupt
            else:
                print("\nTest set not defined or found in dataset YAML. Skipping test set evaluation.")

        except Exception as e:
            print(f"Error loading or evaluating best model: {e}")

    else:
        print(f"Error: Best model weights not found at {best_model_path}")
elif results is None:
    print("Skipping evaluation because training did not run or failed.")
else:
    print("Skipping evaluation because training results object did not contain expected 'save_dir'.")




In [ ]:
# =============================================================================
# ## 7. Next Steps / Further Analysis
#
# Consider:
# *   Analyzing the generated plots (e.g., confusion matrix, PR curves)
#     in the `results.save_dir` (if available). The specific directory is:
#     `{output_config.get('project_name')}/{output_config.get('experiment_name')}`
# *   Performing inference on new images using the trained model
#     (`best_model.predict(...)` if evaluation ran successfully).
# *   Adjusting hyperparameters in `config.yaml` and re-running the training.
# *   Switching datasets by changing `dataset_yaml_path` in `config.yaml`.
# =============================================================================
print("\n--- Script Finished ---")
if results and hasattr(results, 'save_dir'):
    print(f"Check the output directory for detailed results and plots: {results.save_dir}")
elif output_config.get('project_name') and output_config.get('experiment_name'):
     potential_dir = os.path.join(output_config.get('project_name', 'runs/detect'), output_config.get('experiment_name'))
     print(f"Training may have saved results in or near: {potential_dir}")
else:
     print("Output directory information not fully available.")